In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import geopandas as gpd


In [2]:
def load_data(noms: list, path: str, colstokeep: dict = None) -> None:
    for nom in noms:
        chemin_fichier = os.path.join(path, f"{nom}.csv")
        if os.path.exists(chemin_fichier):
            if colstokeep is not None and nom in colstokeep:
                df = pd.read_csv(chemin_fichier, usecols=colstokeep[nom], low_memory=False)
            else:
                df = pd.read_csv(chemin_fichier, low_memory=False)
            globals()[nom] = df
        else:
            print(f"Le fichier {nom+'.csv'} n'existe pas.")

In [3]:
perf_colstokeep = [
    'approche_de_calcul',

    'ift_cible_non_mil_tx_comp',

    'ift_cible_non_mil_chimique_tot',
    'ift_cible_non_mil_chim_tot_hts',
    'ift_cible_non_mil_h',
    'ift_cible_non_mil_f',
    'ift_cible_non_mil_i',
    'ift_cible_non_mil_ts',
    'ift_cible_non_mil_a',
    'ift_cible_non_mil_hh',
    'ift_cible_non_mil_biocontrole',
    'recours_aux_moyens_biologiques',

    'tps_utilisation_materiel',
    'tps_travail_manuel',
    'tps_travail_total',

    'surface_par_unite_de_travail_humain', # noneed
    'nombre_uth_necessaires', # noneed

    'nbre_de_passages_desherbage_meca',
    'utili_desherbage_meca',
    'type_de_travail_du_sol',

    'co_tot_std_mil',
    'cm_std_mil',
    'c_main_oeuvre_tot_std_mil',
    'c_main_oeuvre_tractoriste_std_mil', # noneed
    'c_main_oeuvre_manuelle_std_mil', # noneed

    'conso_carburant',

    'pb_std_mil_avec_autoconso',
    'mb_std_mil_avec_autoconso',
    'msn_reelle_avec_autoconso',
    'md_std_mil_avec_autoconso', # noneed

    'ferti_n_tot',
    'ferti_n_mineral',
    'ferti_n_organique',
    'ferti_p2o5_tot',
    'ferti_p2o5_mineral',
    'ferti_p2o5_organique',
    'ferti_k2o_tot',
    'ferti_k2o_mineral',
    'ferti_k2o_organique',

    'hri1_hts',
    'hri1_g1_hts',
    'hri1_g2_hts',
    'hri1_g3_hts',
    'hri1_g4_hts',

    'qsa_tot_hts',
    'qsa_danger_environnement_hts',
    'qsa_toxique_utilisateur_hts',
    'qsa_cmr_hts',
    'qsa_glyphosate_hts',
    'qsa_cuivre_tot_hts',
    'qsa_soufre_tot_hts'
    ]

alerte_cols = [
    'alerte_ferti_n_tot',
    'alerte_ift_cible_non_mil_chim_tot_hts',
    'alerte_ift_cible_non_mil_f',
    'alerte_ift_cible_non_mil_h',
    'alerte_ift_cible_non_mil_i',
    'alerte_ift_cible_non_mil_biocontrole',
    'alerte_co_irrigation_std_mil',
    'alerte_msn_std_mil_avec_autoconso',
    'alerte_nombre_interventions_phyto',
    'alerte_pb_std_mil_avec_autoconso',
    'alerte_rendement',
    'alertes_charges',
    'alerte_cm_std_mil',
    'alerte_co_semis_std_mil'
]

files = [
    "sdc_realise_performance","synthetise_synthetise_performance",
    
    "sdc","synthetise",
    "plantation_perenne_synthetise","plantation_perenne_realise", "noeuds_realise","noeuds_synthetise", "connection_synthetise","parcelle","zone",
    "composant_culture","variete","espece",
    "domaine","dispositif","commune",

    "intervention_synthetise_agrege","intervention_realise_agrege",    
    "intervention_realise_nettoyage",
    "noeuds_synthetise_restructure","plantation_perenne_synthetise_restructure",#'intervention_synthetise_restructure',
    "identification_pz0",
    "poids_connexions_synthetise_rotation",
    "typologie_can_culture","typologie_can_rotation_synthetise","typologie_assol_can_realise",
    "donnees_spatiales_commune_du_domaine",
    # "action_realise_rendement_total","action_synthetise_rendement_total"
    ]

colstokeep_dict = {
    "sdc_realise_performance": ['sdc_id'] + perf_colstokeep + alerte_cols,
    "synthetise_synthetise_performance": ['synthetise_id'] + perf_colstokeep + alerte_cols,

    "sdc": ['id','code','nom','modalite_suivi_dephy','code_dephy','filiere','type_production','type_agriculture','part_sau_domaine','reseaux_ir','reseaux_it','dispositif_id','validite'],
    "synthetise": ['id', 'nom', 'campagnes', 'sdc_id'],

    "plantation_perenne_synthetise" : ['id','synthetise_id'],
    "plantation_perenne_realise" : ['id','culture_id','zone_id'],
    "noeuds_synthetise" : ['id','synthetise_id'],
    "noeuds_realise" : ['id','culture_id','zone_id'],
    "parcelle": ['id','sdc_id'],
    "zone": ['id','parcelle_id'],

    "espece": ['id','code_espece_botanique','libelle_espece_botanique','typocan_espece','typocan_espece_maraich'],
    "variete" : ['id','denomination'],

    "domaine" : ['id','code','nom','campagne','commune_id','sau_totale'],
    "dispositif": ['id','type','domaine_id'],
    }

In [4]:
path = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_outils'
load_data(files, path, colstokeep_dict)

del(perf_colstokeep, colstokeep_dict, files)

In [5]:
# On combine les performances du réalisé et du synthétisé
df = pd.concat([
    (synthetise_synthetise_performance.merge(synthetise.rename(columns={'id':'synthetise_id', 'campagnes':'synthetise_campagne'}), on='synthetise_id', how='left', suffixes=(None,'_synthetise'))), 
    sdc_realise_performance
])

# On ajoute les infos des sdc, dispositifs et domaines
df = df.merge(sdc.rename(columns={'id':'sdc_id'}), on='sdc_id', how='left', suffixes=(None,'_sdc'))
df = df.merge(dispositif.rename(columns={'id':'dispositif_id'}), on='dispositif_id', how='left', suffixes=(None,'_dispositif'))
df = df.merge(domaine.rename(columns={'id':'domaine_id'}), on='domaine_id', how='left', suffixes=(None,'_domaine'))



In [6]:
# On filtre pour ne garder que les DEPHY FERME en détaillés
df = df.loc[(df['modalite_suivi_dephy']=='DETAILLE') & (df['type']=='DEPHY_FERME')]

# On filtre les sdc sans numéro DEPHY valide
## On nettoie les numéros DEPHY
df['code_dephy'] = df['code_dephy'].astype(str).\
    str.replace('PPZ', '').\
        str.replace('_', '').\
            str.replace('-', '').\
                str.replace(' ', '').\
                    str.replace('\t', '').\
                        str.upper().\
    where(df['code_dephy'].notna(), None)

df['code_dephy'] = np.where(df['code_dephy']=='', None, df['code_dephy'])

## On garde uniquement les codes DEPHY au format exact AAAANNNN ou AAAAXNNN
pattern_exact = r'^[A-Z]{3}(?:[0-9]|X)\d{4}$'
df = df.loc[df['code_dephy'].str.fullmatch(pattern_exact, na=False)]

# On enleve toute les données n'ayant pas de type d'agriculture (AB, Conv, en conversion)
df = df.loc[(df['type_agriculture'].notna()) & (df['type_agriculture'] != 'Information obligatoire')]

In [7]:
# Compter le nombre de "synthétisé" et "réalisé" par sdc_id
counts = df.groupby('sdc_id')['approche_de_calcul'].value_counts().unstack().fillna(0)

# Identifier les sdc_id avec exactement 1 "synthétisé" et 1 ou plus "réalisé"
sdc_ids_to_keep_synthetise = counts[(counts['synthétisé'] == 1) & (counts['réalisé'] >= 1)].index

# Identifier les sdc_id avec au moins 2 "synthétisé"
sdc_ids_with_multiple_synthetise = counts[counts['synthétisé'] >= 2].index

# Filtrer en éliminant les doublons de sdc_id, en priorisant les synthétisés, et lorsqu'il y a plusieurs synthétisés virer ceux dont les camapgnes du synthétisé ne correspondent pas à la campagne du domaine
df = df[
    (
        (df['sdc_id'].isin(sdc_ids_to_keep_synthetise)) &
        (df['approche_de_calcul'] == 'synthétisé')
    ) |
    (
        (df['sdc_id'].isin(sdc_ids_with_multiple_synthetise)) &
        (df['approche_de_calcul'] == 'synthétisé') &
        (df.apply(lambda row: str(row['campagne']) in str(row['synthetise_campagne']).split(', '), axis=1))
    ) |
    (~df['sdc_id'].isin(sdc_ids_with_multiple_synthetise) & ~df['sdc_id'].isin(sdc_ids_to_keep_synthetise))
]

In [8]:
a = len(
    df.loc[(df['approche_de_calcul'] == 'synthétisé') &
        ~(df.apply(lambda row: str(row['campagne']) in str(row['synthetise_campagne']).split(', '), axis=1)),
        ['campagne','synthetise_campagne']]
)
print("nombre de synthétisé dont la campagne du domaine n'est pas dans les campagnes synthétisées :", a)
if a != 0: raise ValueError("Il reste des synthétisés dont la campagne du domaine n'est pas dans les campagnes synthétisées.")
else : print("Ok !")

nombre de synthétisé dont la campagne du domaine n'est pas dans les campagnes synthétisées : 0
Ok !


In [9]:
# Il reste des doublons à gérer (les sdc_id qui ont plusieurs synthétisés, et éventuellement des réalisés)

# On crée un df avec seulement les sdc_id avec encore des doublons
df_still_doubl = df.loc[df.duplicated(subset=['sdc_id'], keep=False)].copy()

df_still_doubl['_ift'] = df_still_doubl['ift_cible_non_mil_tx_comp'].fillna(-float('inf'))
df_still_doubl['_validite'] = df_still_doubl['validite'].fillna(-float('inf'))
df_still_doubl['_len'] = df_still_doubl['synthetise_campagne'].astype(str).str.len()

df_sdc_to_keep = (
    df_still_doubl
    .sort_values(
        by=[
            'sdc_id',
            '_ift',         # On priorise les taux de compéltion d'ift les plus élevés
            '_validite',    # Puis les sdc valides
            '_len',         # Puis les campagnes synthétisés les plus longues
        ],
        ascending=[True, False, False, False], kind='mergesort'
    )
    .groupby('sdc_id', sort=False)
    .head(1)
    .drop(columns=['_ift', '_validite', '_len'])
)


# Maintenant on modifie df pour appliqué les filtres
# Supprimer de df toutes les lignes de ces sdc_id qui étaient dans df_still_doubl mais pas dans df_sdc_to_keep
df = df.loc[~df['synthetise_id'].isin(
    df_still_doubl.loc[~df_still_doubl['synthetise_id'].isin(df_sdc_to_keep['synthetise_id']), 'synthetise_id']
)]

In [10]:
# Définir les années limites, soit celle en cours et la plus vieille acceptable 2008
annee_en_cours = datetime.now().year
annees_trop_vieille = 2004 # des pz0 attendues jusqu'en 2005

# Créer la fonction qui filtre les années au seins d'un synthétisé mutli-annuel
def filtrer_annees(annees_str):
    if pd.isna(annees_str):
        return None
    if not isinstance(annees_str, str): annees_str = str(annees_str)
    annees = [int(a.strip()) for a in annees_str.split(",")]
    annees_filtrees = [str(a) for a in annees if (a < annee_en_cours) & (a > annees_trop_vieille)]
    return ", ".join(annees_filtrees) if annees_filtrees else None

# Appliquer la fonction de filtrage des années aux campagnes en synthétisé
df['synthetise_campagne'] = df['synthetise_campagne'].apply(filtrer_annees)

# Si toutes les années d'un synthétisé ont été filtrées, on supprime la ligne
# Pareil pour les réalisés dont la campagne est trop récente ou trop vieille
df = df.loc[(pd.to_numeric(df['campagne'], errors='coerce') < annee_en_cours) &
            (pd.to_numeric(df['campagne'], errors='coerce') > annees_trop_vieille) &
            (df['synthetise_campagne'].notna())]

In [11]:
# Fonction utile pour convertir une cellule d'une série qui est sous forme de liste en scalaire si elle ne contient qu'une seule valeur unique, ou en None si ellen ne contient aucune valeur unique
def list_to_scalar(serie):
    unique_values = list(serie.dropna().unique())
    if len(unique_values) == 0:
        return None
    elif len(unique_values) == 1:
        return unique_values[0]
    else:
        return unique_values

# Préparer un dataframe des sdc réalisé avec leur pz0 identifié
pz0ident_real = intervention_realise_agrege.merge(identification_pz0.rename(columns={'entite_id':'zone_id'}), on='zone_id', how='left')[['sdc_id','pz0']].groupby('sdc_id')['pz0'].apply(lambda x: list_to_scalar(x), include_groups=False).reset_index()
pz0ident_real = pz0ident_real.loc[pz0ident_real['pz0'].notna()]

if len(pz0ident_real.loc[pz0ident_real['pz0'].apply(lambda x: isinstance(x, list))] ) > 0 :
    raise ValueError("Il y a des sdc réalisé avec plusieurs identification différentes selon leur zones")

# On supprime les synthétisés ou réalisés n'ayant pas de pz0 identifié correctement après avoir merge les identifications
df = df.merge(identification_pz0.rename(columns={'entite_id':'synthetise_id'}), on='synthetise_id', how='left')
df = df.merge(pz0ident_real.rename(columns={'entite_id':'sdc_id'}), on='sdc_id', how='left')
df['pz0'] = df['pz0_x'].combine_first(df['pz0_y'])
df = df.drop(columns=['pz0_x','pz0_y'])

print(f"Il y a {len(df.loc[~df['pz0'].isin(['pz0','post'])])} synthétisé ou sdc filtrés n'ayant pas de pz0 identifié correctement.")

df = df.loc[df['pz0'].isin(['pz0','post'])]

Il y a 3234 synthétisé ou sdc filtrés n'ayant pas de pz0 identifié correctement.


In [12]:
def explode_campagne(df):
    """ 
    Doit contenir les colonnes pz0 + synthetise_campagne + campagne !
    Explose (duplique) les campagnes multi-annuelles en lignes séparées.
    Utilise la colonne 'synthetise_campagne' pour créer la nouvelle colonne de campagne explosée.
    Si on est en réalisé, on prend la colonne 'campagne' pour la nouvelle colonne.
    """
    df['new_campagne'] = df['synthetise_campagne']
    df['new_campagne'] = df['new_campagne'].fillna(df['campagne'])
    df['new_campagne'] = df['new_campagne'].astype(str).str.split(', ')

    # Lorsqu'une entité est un pz0 on ne garde que la dernière année du synthétisé pour ne pas donner plus de poids au point zéro qu'il ne le faudrait (on veut qu'une seule ligne, pas 3). On prendra le max des années dans ce cas.
    df['new_campagne'] = np.where(df['pz0']=='pz0', 
                                  df['new_campagne'].apply(lambda x: str(max(int(year) for year in x if year.strip().isdigit())) if isinstance(x, list) else x),
                                  df['new_campagne'])

    df_exploded = df.explode('new_campagne')
    return df_exploded

# On explode les campagnes multi-annuelles (= on les duplique pour chaque année du synthétisé)
df = explode_campagne(df)

In [13]:
# Supprimer les entités tagué pz0 lorsqu'il y en a un tagué pz0 plus récente (les 3*monoannuels en pérenne)
# Pour les multiannuel c'est deja fait en ne gardant que l'année la plus récente lors de explode_campagne()

mask_pz0 = (df['pz0'] == 'pz0')
df['rank'] = df[mask_pz0].groupby('code_dephy')['new_campagne'].rank(ascending=False, method='first')

df = df[
    ((mask_pz0) & (df['rank'] == 1)) |  # 'pz0' les plus récentes
    (~mask_pz0)                         # toutes les 'post'
].drop(columns=['rank']) 

In [14]:
# Suppression des code_dephy avec une seule et unique campagne
# ainsi que ceux n'ayant auucn pz0 !

def get_ndephy_without_pz0(group):
    if len(group)<=1 :
        return False
    if len(group.loc[group['pz0']=='pz0']) > 1 :
        raise ValueError('Aller check la suppression les pz0 ancien chez les 3*mono-annuels')
    if any(group['pz0']=='pz0')== False :
        return False
    return True

mask_dephy_to_keep = df.groupby('code_dephy').apply(get_ndephy_without_pz0,include_groups=False).reset_index()
mask_dephy_to_keep = mask_dephy_to_keep.loc[mask_dephy_to_keep[0], 'code_dephy'].to_list()

df = df.loc[df['code_dephy'].isin(mask_dephy_to_keep)]

In [15]:
# On détecte les espèces et variétés principales présentes dans les sdc
# On ne garde que s'il y a une et une seule espèce ou variété possible dans le sdc/synthétisé

load_data(['TEST_species_variety'], path) # = get_all_species_and_variety_from_sdc_or_synth(donnees)

# Espece arboriculture
TEST_species_variety['c111_species'] = np.where((
    TEST_species_variety['filiere']=='ARBORICULTURE') & (TEST_species_variety['size_unique_sp']==1),
    TEST_species_variety['unique_species_util'], np.nan)
# Variete arboriculture
TEST_species_variety['c112_variety'] = np.where((
    TEST_species_variety['filiere']=='ARBORICULTURE') & (TEST_species_variety['size_unique_var']==1),
    TEST_species_variety['unique_variete_util'], np.nan)

# Variété viticulture
TEST_species_variety['c113_grapeVar'] = np.where((
    TEST_species_variety['filiere']=='VITICULTURE') & (TEST_species_variety['size_unique_var']==1),
    TEST_species_variety['unique_variete_util'], np.nan)

df['group_id'] = df['synthetise_id'].fillna(df['sdc_id'])
df = df.merge(
    TEST_species_variety[['group_id', 'c111_species', 'c112_variety', 'c113_grapeVar']],
    on='group_id',
    how='left'
)

In [16]:
# On crée les typologie de système simplifiés pour DG
def add_type_agri(df, col):
    return np.where((df['type_agriculture'] == 'Agriculture conventionnelle') & 
                     (df[col] != np.nan),
                     df[col] + ' conventionnels', 
                     np.where((df['type_agriculture'].isin(["En conversion vers l'agriculture biologique",
                                                            'Agriculture biologique'])) & 
                              (df[col] != np.nan),
                              df[col] + ' biologiques', 
                              df[col]))

def nan_to_none(serie):
    return np.where((serie.isna())|(serie == 'nan'), None, serie)

# ARBO
df['c120_arboriculture_typo_sdc'] = np.select(
    [
        df['c111_species'].isna(),
        df['c111_species'].isin(['Pommier', 'Poirier']),
        df['c111_species'].isin(['Abricotier', 'Prunier']),
        ~(df['c111_species'].isin(['Pommier', 'Poirier','Abricotier', 'Prunier']))
    ],
    [
        np.nan,
        'Pommiers et Poiriers',
        'Abricotiers et Pruniers',
        df['c111_species'] + 's'
    ],
    default=np.nan)

df['c120_arboriculture_typo_sdc'] = add_type_agri(df, 'c120_arboriculture_typo_sdc')
df['c120_arboriculture_typo_sdc'] = np.where(df['filiere']=='ARBORICULTURE',
                                    df['c120_arboriculture_typo_sdc'],
                                    None) # vérification variable que en ARBO
df['c120_arboriculture_typo_sdc'] = nan_to_none(df['c120_arboriculture_typo_sdc'])

# MARAICH
df['c121_maraichage_typo_sdc'] = df['type_production'].str.lower().str.capitalize().str.replace('_', '-')
df['c121_maraichage_typo_sdc'] = add_type_agri(df, 'c121_maraichage_typo_sdc')
df['c121_maraichage_typo_sdc'] = np.where(df['filiere']=='MARAICHAGE',
                                    df['c121_maraichage_typo_sdc'],
                                    None) # vérification variable que en MARAICH
df['c121_maraichage_typo_sdc'] = nan_to_none(df['c121_maraichage_typo_sdc'])


# HORTICULTURE
df['c122_horticulture_typo_sdc']  = np.select(
    [
        df['type_production'].isna(),
        df['type_production'] == 'CONTAINER_NURSERY',
        df['type_production'] == 'OPEN_GROUND_NURSERY',
        df['type_production'] == 'MIX',
        df['type_production'] == 'POT_PLANTS'
    ],
    [
        np.nan,
        'Pépinière hors sol, en conteneur',
        'Pépinières pleine terre',
        'Pépinières mixte',
        'Plantes en pot'
    ],
    default=np.nan)
df['c122_horticulture_typo_sdc'] = np.where(df['filiere']=='HORTICULTURE',
                                    df['c122_horticulture_typo_sdc'],
                                    None) # vérification variable que en HORTI
df['c122_horticulture_typo_sdc'] = nan_to_none(df['c122_horticulture_typo_sdc'])


# TROP
# On ajoutera les données des cultures tropicales à la fin avec un Excel envoyé par la CAN mais il nous faut une colonne même vide pour l'instant afin de faire le bind.
df['c123_cult_tropicales_typo_sdc'] = None


# GCPE
df['c124_gcpe_typo_sdc'] = df['filiere'].str.lower().str.replace('_e', ' é').str.capitalize().str.replace('_', ' ')
df['c124_gcpe_typo_sdc'] = add_type_agri(df, 'c124_gcpe_typo_sdc')
df['c124_gcpe_typo_sdc'] = np.where(df['filiere'].isin(['POLYCULTURE_ELEVAGE','GRANDES_CULTURES']),
                                    df['c124_gcpe_typo_sdc'],
                                    None) # vérification variable que en GCPE
df['c124_gcpe_typo_sdc'] = nan_to_none(df['c124_gcpe_typo_sdc'])

In [17]:
# Ajout des données géographiques

path_geo_data = "/home/administrateur/Bureau/Datagrosyst/catalogue_script_agrosyst/02_outils/data/external_data/geospatial_data/" # TODO revoir le path

arrond_data = gpd.read_file(path_geo_data + "geoVec_arrond.json") 
arrond_data['arrondissement_code'] = arrond_data['Arrondissement_code_name'].str.split(' - ', expand=True)[0]
arrond_data = arrond_data[['arrondissement_code','Arrondissement_code_name']].rename(columns={'Arrondissement_code_name':'arrondissement'})

dep_data = gpd.read_file(path_geo_data + "geoVec_dep.json") 
dep_data['departement_code'] = dep_data['Department_code_name'].str.split(' - ', expand=True)[0]
dep_data = dep_data[['departement_code','Department_code_name']].rename(columns={'departement_code':'departement'})

commune = commune.merge(arrond_data, on='arrondissement_code', how='left')
commune = (commune.merge(dep_data, on='departement', how='left')
           .drop(columns='departement')
           .rename(columns={'Department_code_name':'departement'}))

df = df.merge(commune[['id','codeinsee', 'departement', 'region', 'bassin_viticole', 'ancienne_region','latitude', 'longitude','arrondissement']].rename(columns={'id':'commune_id'}), on='commune_id', how='left')

In [18]:
# Calcul de l'IFT hors herbicide et hors TS
df['c602_IFT_hh_hts'] = df['ift_cible_non_mil_hh'] - df['ift_cible_non_mil_ts']

# Lucie avait ajouter en variable masquée "c640_stdTotalIFT", l'IFT normé. Pas besoin pour l'instant
df['c504_outLabourTotalExpenses'] = df['co_tot_std_mil'] + df['cm_std_mil']

In [20]:
dict_var_impacted = {
    'nbre_de_passages_desherbage_meca': 
				[None],
	'tps_travail_total': 
				['tps_utilisation_materiel', 'tps_travail_manuel'],
	'tps_utilisation_materiel': 
				['tps_travail_total'],
	'tps_travail_manuel': 
				['tps_travail_total'],
	'conso_carburant': 
				[None],
                

	'ferti_n_tot': 
				['ferti_n_mineral', 'ferti_n_organique'],
	'ferti_n_mineral': 
				['ferti_n_tot'],
	'ferti_n_organique': 
				['ferti_n_tot'],
	'ferti_p2o5_tot': 
				['ferti_p2o5_mineral', 'ferti_p2o5_organique'],
	'ferti_p2o5_mineral': 
				['ferti_p2o5_tot'],
	'ferti_p2o5_organique': 
				['ferti_p2o5_tot'],
	'ferti_k2o_tot': 
				['ferti_k2o_mineral', 'ferti_k2o_organique'],
	'ferti_k2o_mineral': 
				['ferti_k2o_tot'],
	'ferti_k2o_organique': 
				['ferti_k2o_tot'],
                

	'pb_std_mil_avec_autoconso': 
				['mb_std_mil_avec_autoconso', 'msn_reelle_avec_autoconso','md_std_mil_avec_autoconso'],
	'mb_std_mil_avec_autoconso': 
				['msn_reelle_avec_autoconso', 'co_tot_std_mil', 'c504_outLabourTotalExpenses','md_std_mil_avec_autoconso'],
	'msn_reelle_avec_autoconso': 
				['pb_std_mil_avec_autoconso', 'mb_std_mil_avec_autoconso', 'co_tot_std_mil', 'cm_std_mil', 'c504_outLabourTotalExpenses','md_std_mil_avec_autoconso'],
	'c504_outLabourTotalExpenses': 
				['msn_reelle_avec_autoconso', 'cm_std_mil', 'co_tot_std_mil','md_std_mil_avec_autoconso'],
	'co_tot_std_mil': 
				['mb_std_mil_avec_autoconso', 'msn_reelle_avec_autoconso', 'c504_outLabourTotalExpenses','md_std_mil_avec_autoconso'],
	'c_main_oeuvre_tot_std_mil': 
				['md_std_mil_avec_autoconso'],
	'cm_std_mil': 
				['msn_reelle_avec_autoconso', 'c504_outLabourTotalExpenses','md_std_mil_avec_autoconso'],
    'md_std_mil_avec_autoconso':
				[None], # si les indicateur éco sont mauvais, on passe la marge directe en NA. L'inverse, on verra plus tard, quand la marge directe sera validée par la CAN                


	'ift_cible_non_mil_chim_tot_hts': 
				['c602_IFT_hh_hts', 'ift_cible_non_mil_a', 'ift_cible_non_mil_i', 'ift_cible_non_mil_f', 'ift_cible_non_mil_h', 
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'c602_IFT_hh_hts': 
				['ift_cible_non_mil_chim_tot_hts', 'ift_cible_non_mil_a', 'ift_cible_non_mil_i', 'ift_cible_non_mil_f', 
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'ift_cible_non_mil_biocontrole': 
				['qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'ift_cible_non_mil_h': 
				['ift_cible_non_mil_chim_tot_hts',
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'ift_cible_non_mil_i': 
				['ift_cible_non_mil_chim_tot_hts', 'c602_IFT_hh_hts', 
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'ift_cible_non_mil_f': 
				['ift_cible_non_mil_chim_tot_hts', 'c602_IFT_hh_hts', 
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'ift_cible_non_mil_a': 
				['ift_cible_non_mil_chim_tot_hts', 'c602_IFT_hh_hts', 
     			'qsa_tot_hts', 'qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'recours_aux_moyens_biologiques': 
				[None],
                

	'hri1_hts': 
				['hri1_g1_hts','hri1_g2_hts','hri1_g3_hts','hri1_g4_hts'],
	'hri1_g1_hts': 
				[None],
	'hri1_g2_hts': 
				[None],
	'hri1_g3_hts': 
				[None],
	'hri1_g4_hts': 
				[None],
    

	'qsa_tot_hts': 
				['qsa_danger_environnement_hts', 'qsa_toxique_utilisateur_hts','qsa_cmr_hts','qsa_glyphosate_hts','qsa_cuivre_tot_hts','qsa_soufre_tot_hts'],
	'qsa_cmr_hts': 
				[None],
	'qsa_toxique_utilisateur_hts': 
				[None],
	'qsa_glyphosate_hts': 
				[None],
	'qsa_danger_environnement_hts': 
				[None],
	'qsa_cuivre_tot_hts': 
				[None],
	'qsa_soufre_tot_hts': 
				[None]
}

In [22]:
alerte_col = ['alerte_ferti_n_tot',
'alerte_ift_cible_non_mil_chim_tot_hts',
'alerte_ift_cible_non_mil_f',
'alerte_ift_cible_non_mil_h',
'alerte_ift_cible_non_mil_i',
'alerte_ift_cible_non_mil_biocontrole',
'alerte_co_irrigation_std_mil',
'alerte_msn_std_mil_avec_autoconso',
'alerte_nombre_interventions_phyto',
'alerte_pb_std_mil_avec_autoconso',
'alerte_rendement',
'alertes_charges',
'alerte_cm_std_mil',
'alerte_co_semis_std_mil']

In [27]:
with open('resultats_alertes.txt', 'w') as fichier:
    for col in alerte_col:
        fichier.write(f"Colonne : {col}\n")
        fichier.write("\n")
        result = df[col].fillna('nan').reset_index().groupby(col).size().sort_values(ascending=False)
        fichier.write(result.to_string())
        fichier.write("\n\n")
        fichier.write("------------------------")
        fichier.write("\n")

In [ ]:
# Filtres OUTLIERS

def make_na_if_warnings(df, variable_col, warning_col):
    list_warn_ok = ["Pas d'alerte"]
    df[df[warning_col]]


for quanti_col in list(dict_var_impacted.keys()) :
    if quanti_col == 'ferti_n_tot' :


alerte_ferti_n_tot
alerte_ift_cible_non_mil_chim_tot_hts
alerte_ift_cible_non_mil_f
alerte_ift_cible_non_mil_h
alerte_ift_cible_non_mil_i
alerte_ift_cible_non_mil_biocontrole
alerte_co_irrigation_std_mil
alerte_msn_std_mil_avec_autoconso
alerte_nombre_interventions_phyto
alerte_pb_std_mil_avec_autoconso
alerte_rendement
alertes_charges
alerte_cm_std_mil
alerte_co_semis_std_mil

In [ ]:
# On enleve toutes les ligne où l'espèce principale est vigne, autre que dans la filière VITI
df = df.loc[~((df['c111_species']=='Vigne') & (df['c101_sector']!='VITICULTURE'))]

# Filtre semis direct en GCPE
# Il est fortement peu probable qu’un sdc soit en Semis Direct s’il est en Agriculture Biologique. Lorsque c’est le cas on transforme Semis Direct de c201_groundWorkType en NA, car on part du principe que le type de travail du sol est moins fiable que le type de conduite du sdc.
df.loc[(df['c201_groundWorkType'] == 'SEMIS_DIRECT') &
       (df['c101_sector'].isin(['GRANDES_CULTURES','POLYCULTURE_ELEVAGE'])) &
       (df['c110_managementType'].isin(["En conversion vers l'agriculture biologique",'Agriculture biologique'])),
       'c201_groundWorkType'] = np.nan

# Ce filtre fonctionne comme le principe du chapitre “Filtres outliers” sauf qu’il se base sur un groupe utilisant la variable c201_groundWorkType. Si un IFT herbicide (c605_herbicideIFT) est strictement inférieur à 0.5 en GCPE alors que son type de travail du sol renseigné est Semis direct alors on transforme en NA les valeurs des variables c605_herbicideIFT et c601_totalIFT.
list_var_a_passer_en_na = ['c605_herbicideIFT','c601_totalIFT']
df.loc[(df['c201_groundWorkType'] == 'SEMIS_DIRECT') &
       (df['c101_sector'].isin(['GRANDES_CULTURES','POLYCULTURE_ELEVAGE'])) &
       (df['c605_herbicideIFT'] < 0.5),
       list_var_a_passer_en_na] = np.nan

# On veut absolument un valeur pour longitude/latitude/realized/sdc_id. On filtre les lignes qui n'ont pas une des quatre variables
df = df.loc[~((df['longitude'].isna()) | (df['latitude'].isna()) | (df['realized'].isna()) | (df['sdc_id'].isna()))]

In [ ]:
# Calcul des évolutions : Nombre d'année dans le réseau & Différences d'IFT entre pz0 et post-pz0
# On calcul les évolution APRES la filtration des données abérrentes, pour ne calculer que les évolutions vraissemblables

def get_evol(group, col):
    mask_pz0 = group['c102_pz0'] == 'pz0'

    if mask_pz0.sum() != 1:
        raise ValueError('Groupe dephy sans ou avec trop de pz0')

    if col == 'c104_campaign_dis':
        pz0_val = int(group.loc[mask_pz0, col].iloc[0])
        result = np.where(
            mask_pz0 | group[col].isna(),
            np.nan,
            group[col].astype(int) - pz0_val
        )
        result = pd.Series(result, index=group.index)
        result = result.map(lambda x: (
                            np.nan if pd.isna(x)
                            else (f"pz0 + {int(x):02d} an" if int(x) == 1
                                else f"pz0 + {int(x):02d} ans")))
        return result

    else:
        pz0_val = group.loc[mask_pz0, col].iloc[0]
        result = np.where(
            mask_pz0 | group[col].isna(),
            np.nan,
            group[col] - pz0_val
        )
        return pd.Series(result, index=group.index)

def apply_evol(group):
    group = group.copy()

    group['c103_networkYears'] = get_evol(group, 'c104_campaign_dis')
    group['c701_totalIFT_evol_diff'] = get_evol(group, 'c601_totalIFT')
    group['c702_IFT_hh_hts_evol_diff'] = get_evol(group, 'c602_IFT_hh_hts')
    group['c703_biocontrolIFT_evol_diff'] = get_evol(group, 'c603_biocontrolIFT')
    group['c705_herbicideIFT_evol_diff'] = get_evol(group, 'c605_herbicideIFT')
    group['c707_insecticideIFT_evol_diff'] = get_evol(group, 'c607_insecticideIFT')
    group['c708_fungicideIFT_evol_diff'] = get_evol(group, 'c608_fungicideIFT')
    group['c709_otherIFT_evol_diff'] = get_evol(group, 'c609_otherIFT')
    group['c710_biologicalWaysSolution_evol_diff'] = get_evol(group, 'c610_biologicalWaysSolution')

    return group

df = df.groupby('dephyNb').apply(apply_evol).reset_index(drop=True)

In [ ]:
# On regarde les les duplicat par numéro DEPHY et par années
# En regardant les années une fois dupliquées (explode)
# A faire après le calcul des évolution car nous allons modifier les numéro DEPHY de certain, or nous avons besoin de grouper par numéro DEPHY dans le calcul des évolutions

print(f"Après les filtres précédents et le explode des années, il reste {len(df.loc[df.duplicated(subset=['dephyNb','c104_campaign_dis'], keep=False)])} entitées en doublons au vu de la campagne et du code DEPHY. Ce sont des cas où les données sont légitimes mais le numéro DEPHY est dupliqué, par exemple, un sdc_id pour la ferme lorsque le sdc est en Bio et un lorsqu'il est en conventionnel. Devrait avoir un numéro DEPHY pour chaque mais pour DEPHY nous gardons ces bonnes données en modifiant légèrement le numéro DEPHY")

# Modification des numéro DEPHY lorsqu'il y a duplicat de numéro_DEPHY*annee
# On fait un tri par le nom du sdc pour que lorsqu'il y ai duplicat de numéro_DEPHY*annee, le _0 de l'année n soit également en _0 en année n+1 (et pas _0 > _1 vs son duplicat qui passe de _1 > _0). On veut une cohénrence de frise.
df = df.sort_values(by='nom_sdc')
mask = df.duplicated(subset=['dephyNb','c104_campaign_dis'], keep=False)
df.loc[mask, 'dephyNb'] = (
    df.loc[mask, 'dephyNb'] + '_' +
    (df.loc[mask].groupby(['dephyNb','c104_campaign_dis']).cumcount()).astype(str)
    )
df = df.sort_values(by=['dephyNb','c104_campaign_dis'])

In [ ]:
# Disponibilité par filière
gcpe = ['GRANDES_CULTURES','POLYCULTURE_ELEVAGE']

df.loc[~df['c101_sector'].isin(gcpe), 
       ['c201_groundWorkType', 'c203_mechanicalWeeding', 'c204_mechanicalWeedingInterventionFrequency', 'c501_grossProceeds', 'c502_grossProfit', 'c503_semiNetMargin']] = np.nan
df.loc[~df['c101_sector'].isin(['ARBORICULTURE','VITICULTURE','MARAICHAGE','CULTURES_TROPICALES']), 
       ['c303_labourTime']] = np.nan
df.loc[df['c101_sector']!='ARBORICULTURE', 
       ['c111_species','c112_variety']] = np.nan
df.loc[df['c101_sector']!='VITICULTURE', 
       ['c113_grapeVar']] = np.nan

# Regle du 2021.11.21
# Enlever les données relatifs au fichier “vartohide” du 21/11/2021. Les experts filières, doutant de leur données sur certains type de variables, préfèrent cacher les variables problématiques.
df.loc[~df['c101_sector'].isin(gcpe + ['VITICULTURE']),  
       ['c301_workingTime']] = np.nan
df.loc[~df['c101_sector'].isin(gcpe), 
       ['c401_fertilizationUnityN', 'c402_fertilizationMineralUnityN', 'c403_fertilizationOrganicUnityN',
        'c404_fertilizationUnityP', 'c405_fertilizationMineralUnityP', 'c406_fertilizationOrganicUnityP',
        'c407_fertilizationUnityK', 'c408_fertilizationMineralUnityK', 'c409_fertilizationOrganicUnityK',
        'c302_mechanizationTime',
        'c504_outLabourTotalExpenses', 'c505_operatingExpenses','c507_mechanizationExpenses']] = np.nan

In [ ]:
# Renommer les colonnes et ajouter les nécessaires

# 1. Renommer les colonnes
df = df.rename(columns={
    'synthetise_id' 				            : "systeme_synthetise_id",
	'approche_de_calcul' 				        : "realized",
	# 'ift_cible_non_mil_chimique_tot' 			: "aaaaa",
    'ift_cible_non_mil_chim_tot_hts' 			: "c601_totalIFT",
	'ift_cible_non_mil_h' 				        : "c605_herbicideIFT",
    'ift_cible_non_mil_f' 				        : "c608_fungicideIFT",
	'ift_cible_non_mil_i' 				        : "c607_insecticideIFT",
	# 'ift_cible_non_mil_ts' 				    : "aaaaa",
    'ift_cible_non_mil_a' 				        : "c609_otherIFT",
	# 'ift_cible_non_mil_hh' 				    : "aaaaa",
    'ift_cible_non_mil_biocontrole' 			: "c603_biocontrolIFT",
	# 'ift_cible_non_mil_tx_comp' 				: "aaaaa",
    'recours_aux_moyens_biologiques' 			: "c610_biologicalWaysSolution",
    # 'c701_totalIFT_evol_diff'		            : "aaaaa" # Garde le même nom
    # 'c702_IFT_hh_hts_evol_diff'		        : "aaaaa" # Garde le même nom
    # 'c703_biocontrolIFT_evol_diff'		    : "aaaaa" # Garde le même nom
    # 'c705_herbicideIFT_evol_diff'		        : "aaaaa" # Garde le même nom
    # 'c707_insecticideIFT_evol_diff'		    : "aaaaa" # Garde le même nom
    # 'c708_fungicideIFT_evol_diff'		        : "aaaaa" # Garde le même nom
    # 'c709_otherIFT_evol_diff'		            : "aaaaa" # Garde le même nom
	'tps_utilisation_materiel' 			    	: "c302_mechanizationTime",
    'tps_travail_manuel' 				        : "c303_labourTime",
	'tps_travail_total' 				        : "c301_workingTime",
    'nbre_de_passages_desherbage_meca' 			: "c204_mechanicalWeedingInterventionFrequency",
	'utili_desherbage_meca' 			       	: "c203_mechanicalWeeding",
    'type_de_travail_du_sol' 			    	: "c201_groundWorkType",
	'co_tot_std_mil' 			            	: "c505_operatingExpenses",
	'cm_std_mil' 				                : "c507_mechanizationExpenses",
    'c_main_oeuvre_tot_std_mil' 				: "c506_labourExpenses",
	# 'c_main_oeuvre_tractoriste_std_mil' 		: "aaaaa",
    # 'c_main_oeuvre_manuelle_std_mil' 			: "aaaaa",
	'pb_std_mil_avec_autoconso' 				: "c501_grossProceeds",
    'mb_std_mil_avec_autoconso' 				: "c502_grossProfit",
	'msn_reelle_avec_autoconso' 				: "c503_semiNetMargin",
    # 'c504_outLabourTotalExpenses'				: "aaaaa" # Garde le même nom
	'md_std_mil_avec_autoconso' 				: "c508_DirectMargin", # A ajouter en invisible
	# 'surface_par_unite_de_travail_humain' 	: "aaaaa",
    # 'nombre_uth_necessaires' 				    : "aaaaa",
	'conso_carburant' 				            : "c304_fuelConsumption",
	'ferti_n_tot' 				                : "c401_fertilizationUnityN",
    'ferti_n_mineral' 			            	: "c402_fertilizationMineralUnityN",
	'ferti_n_organique' 				        : "c403_fertilizationOrganicUnityN",
	'ferti_p2o5_tot' 	            			: "c404_fertilizationUnityP",
    'ferti_p2o5_mineral' 	        			: "c405_fertilizationMineralUnityP",
	'ferti_p2o5_organique' 		        		: "c406_fertilizationOrganicUnityP",
	'ferti_k2o_tot' 				            : "c407_fertilizationUnityK",
    'ferti_k2o_mineral' 			        	: "c408_fertilizationMineralUnityK",
	'ferti_k2o_organique' 				        : "c409_fertilizationOrganicUnityK",
	'hri1_hts' 				                    : "c633_hri1tothts",
	'hri1_g1_hts' 			                	: "c634_hri1g1hts",
    'hri1_g2_hts' 			                	: "c635_hri1g2hts",
	'hri1_g3_hts' 			                	: "c636_hri1g3hts",
	'hri1_g4_hts' 		                		: "c637_hri1g4hts",
	'qsa_tot_hts' 			                	: "c640_qteSubstActive", # A ajouter en invisible
    'qsa_danger_environnement_hts' 				: "c644_qteSubstdangerenvironnement",
	'qsa_toxique_utilisateur_hts' 				: "c642_qteSubsttoxique",
    'qsa_cmr_hts' 				                : "c641_qteSubstCMR",
	'qsa_glyphosate_hts' 			            : "c643_qteSubstglyphosate",
	'qsa_cuivre_tot_hts' 				        : "c645_qteSubstcuivre",
    'qsa_soufre_tot_hts' 			          	: "c646_qteSubstsoufre",
	# 'nom' 				                    : "aaaaa",
	# 'synthetise_campagne' 				    : "aaaaa",
	# 'sdc_id' 				                    : "aaaaa", # Garde le même nom
	# 'code' 				                    : "aaaaa",
    # 'nom_sdc' 				                : "aaaaa",
	# 'modalite_suivi_dephy' 			        : "aaaaa",
	'code_dephy' 				                : "dephyNb",
	# 'validite' 				                : "aaaaa",
	'filiere' 				                    : "c101_sector",
    # 'type_production' 			            : "aaaaa",
	'type_agriculture' 			            	: "c110_managementType",
	'part_sau_domaine' 			            	: "aaaaa",
    'dispositif_id' 			            	: "aaaaa",
	# 'reseaux_ir' 				                : "aaaaa", # A laisser pour DEPHY dans Ags-team
	# 'reseaux_it' 			                	: "aaaaa", # A laisser pour DEPHY dans Ags-team
	# 'type' 				                    : "aaaaa",
	# 'domaine_id' 			                	: "aaaaa",
    # 'code_domaine' 				            : "aaaaa",
	# 'nom_domaine' 			                : "aaaaa",
	# 'campagne' 			                    : "aaaaa",
	# 'commune_id' 		                		: "aaaaa",
	# 'sau_totale' 		                		: "aaaaa",
    'pz0' 				                        : "c102_pz0",
	'new_campagne' 		                		: "c104_campaign_dis",
    # 'c103_networkYears'						: "aaaaa", # Garde le même nom
	# 'group_id' 				                : "aaaaa",
	# 'c111_species' 			                : "aaaaa", # Garde le même nom
	# 'c112_variety' 			                : "aaaaa", # Garde le même nom
    # 'c113_grapeVar' 			            	: "aaaaa", # Garde le même nom
	# 'c120_arboriculture_typo_sdc'             : "aaaaa", # Garde le même nom
    # 'c121_maraichage_typo_sdc' 		        : "aaaaa", # Garde le même nom
	# 'c122_horticulture_typo_sdc' 				: "aaaaa", # Garde le même nom
    # 'c123_cult_tropicales_typo_sdc' 	 		: "aaaaa", # Garde le même nom
	# 'c124_gcpe_typo_sdc' 				        : "aaaaa", # Garde le même nom
	# 'codeinsee' 				                : "aaaaa",
    'departement' 				                : "c107_departement",
	'region' 				                    : "c105_administrativeRegion",
	'bassin_viticole' 			            	: "c108_wineBasin",
	'ancienne_region' 			            	: "c106_regionPre2015",
    # 'latitude' 				                : "aaaaa", # Garde le même nom
	# 'longitude' 			                	: "aaaaa", # Garde le même nom
	# 'c602_IFT_hh_hts' 			            : "aaaaa" # Garde le même nom
})

# 2. Ajout des colonnes dont on a besoin pour IPM ou même pour DG
df['validation'] = True
df['c100_networksource'] = "DEPHY Ferme"
df['c1001_networksource'] = "DEPHY Ferme"
df['c109_country'] = "France"

df['c9303_labourTime'] = 		None # df['c303_labourTime']
df['c9502_grossProfit'] = 		None # df['c502_grossProfit']
df['c9503_semiNetMargin'] = 	None # df['c503_semiNetMargin']
df['c9506_labourExpenses'] = 	None # df['c506_labourExpenses']
df['c9633_hri1tothts_ipm'] = 	None # df['c633_hri1tothts']
df['c9634_hri1g1hts_ipm'] = 	None # df['c634_hri1g1hts']
df['c9635_hri1g2hts_ipm'] = 	None # df['c635_hri1g2hts']
df['c9636_hri1g3hts_ipm'] = 	None # df['c636_hri1g3hts']
df['c9637_hri1g4hts_ipm'] = 	None # df['c637_hri1g4hts']


# 3. Sélectionner les colonnes
list_needed_columns = [
	'dephyNb',
	'latitude',
	'longitude',
	'realized',
	'sdc_id',
	'systeme_synthetise_id',
	'validation',
	'arrondissement',
    'reseaux_ir', # metadata : que Agsteam
    'reseaux_it', # metadata : que Agsteam
	'c100_networksource',
	'c1001_networksource',
	'c101_sector',
	'c102_pz0',
	'c104_campaign_dis',
	'c105_administrativeRegion',
	'c106_regionPre2015',
	'c107_departement',
	'c108_wineBasin',
	'c109_country',
	'c110_managementType',
	'c111_species',
	'c112_variety',
	'c113_grapeVar',
	'c120_arboriculture_typo_sdc',
	'c121_maraichage_typo_sdc',
	'c122_horticulture_typo_sdc',
	'c123_cult_tropicales_typo_sdc',
	'c124_gcpe_typo_sdc',
	'c201_groundWorkType',
	'c203_mechanicalWeeding',
	'c204_mechanicalWeedingInterventionFrequency',
	'c301_workingTime',
	'c302_mechanizationTime',
	'c303_labourTime',
	'c304_fuelConsumption',
	'c401_fertilizationUnityN',
	'c402_fertilizationMineralUnityN',
	'c403_fertilizationOrganicUnityN',
	'c404_fertilizationUnityP',
	'c405_fertilizationMineralUnityP',
	'c406_fertilizationOrganicUnityP',
	'c407_fertilizationUnityK',
	'c408_fertilizationMineralUnityK',
	'c409_fertilizationOrganicUnityK',
	'c501_grossProceeds',
	'c502_grossProfit',
	'c503_semiNetMargin',
	'c504_outLabourTotalExpenses',
	'c505_operatingExpenses',
	'c506_labourExpenses',
	'c507_mechanizationExpenses',
    'c508_DirectMargin', # metadata : que pour Agsteam
	'c601_totalIFT',
	'c602_IFT_hh_hts',
	'c603_biocontrolIFT',
	'c605_herbicideIFT',
	'c607_insecticideIFT',
	'c608_fungicideIFT',
	'c609_otherIFT',
	'c610_biologicalWaysSolution',
	'c633_hri1tothts',
	'c634_hri1g1hts',
	'c635_hri1g2hts',
	'c636_hri1g3hts',
	'c637_hri1g4hts',
	'c640_qteSubstActive', # metadata : remplace IFTnormé, ne mettre dispo que sur Agsteam
	'c641_qteSubstCMR',
	'c642_qteSubsttoxique',
	'c643_qteSubstglyphosate',
	'c644_qteSubstdangerenvironnement',
	'c645_qteSubstcuivre',
	'c646_qteSubstsoufre',
	'c9303_labourTime',
	'c9502_grossProfit',
	'c9503_semiNetMargin',
	'c9506_labourExpenses',
	'c9633_hri1tothts_ipm',
	'c9634_hri1g1hts_ipm',
	'c9635_hri1g2hts_ipm',
	'c9636_hri1g3hts_ipm',
	'c9637_hri1g4hts_ipm'
]


In [ ]:
# Modifier les modalités
# TODO a faire en SQL non ? dailleurs on fait le renommage des col en SQL aussi ?

In [ ]:
# Ajouter/Remplacer les données Cultures tropicales
# TODO en sql ?

In [ ]:
# Ajouter les données IPM
# TODO en sql ?